In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from core import EV_Data
%load_ext autoreload
%autoreload 2
# Load the .mat file (assuming 'data_path' is defined and points to your .mat file)
%reload_ext autoreload

In [ ]:
path1 = "/home/ccanodom/data/PyBamm/testing.csv"
path2 = "test_degradation.csv"
path3 = "test_degradation2.csv"
path4 = "test_degradation3.csv"
path5 = "/home/ccanodom/scratch/testing_benchmark.csv"
df = pd.read_csv(path5)


In [ ]:
df.columns

In [ ]:
df["Cycle"].tail(300)

In [ ]:
import plotly.express as px
df_filter = df[df["Cycle"] == 100.0]

px.line(df_filter, x=f"Time", y=f"V", color="Step",height=900)

In [ ]:
df = df[(df["Time"] > 20) & (df["Time"] < 980)]

In [ ]:
df.plot(x = "Time", y = ["D_cap"], subplots = True)

In [ ]:
df.plot(x = "Time", y = ["V", "SoC", "Step", "Cycle", "D_cap"], subplots = True)

In [ ]:
df.plot(x = "Time", y = ["V", "SoC", "Step", "C"], subplots = True, xlim = [100, 120])

In [ ]:
filtered_df = df[df["Step"] == 5]

In [ ]:
filtered_df.plot(x = "Time", y = ["V", "SoC", "Step"], subplots = True, xlim = [100, 120])

In [ ]:
import matplotlib.pyplot as plt
filtered_df = df[df["Step"] == 5]
plt.scatter(filtered_df['SoC'], filtered_df['VoC'], c=filtered_df['Time'], alpha = 0.9, s = 0.3)
# Add labels and title
plt.title('V vs SoC Step 5')  # Plot Title
plt.xlabel('SoC (%)')  # X-axis Label
plt.ylabel('Voltage (V)')  # Y-axis Label
# Display the plot
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Assuming df is already filtered for the USSD discharging phase
# Group the DataFrame by 'Cycle_Number' for plotting
filtered_df = df[df["Step"] == 2]

grouped_by_cycle = filtered_df.groupby('Cycle')



# Assuming step_7_df and grouped_by_cycle are defined as before
# Define the Greys colormap
cmap = cm.get_cmap('Reds')

# Get the total number of cycles to normalize the cycle number to [0, 1]
num_cycles = len(grouped_by_cycle)
norm = mcolors.Normalize(vmin=0, vmax=num_cycles)

# Create a plot
plt.figure(figsize=(12, 8))

# Plot Voltage vs. Time for each cycle
for cycle_number, group in grouped_by_cycle:
    # Reset index to access the 'Time' column
    group = group.reset_index()
    # Adjust time so that each cycle's Step 7 starts at 0
    adjusted_time = group['Time'] - group['Time'].min()
    # Normalize the cycle_number for color selection
    color = cmap(norm(cycle_number))
    plt.plot(adjusted_time, group['V'], label=f'Cycle {cycle_number}', color=color, linewidth=0.1)

plt.title('Voltage vs. Time for Step 9 by Cycle')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Assuming df is already filtered for the USSD discharging phase
# Group the DataFrame by 'Cycle_Number' for plotting
filtered_df = df[df["Step"] == 5]

grouped_by_cycle = filtered_df.groupby('Cycle')




# Define the colormap
cmap = cm.get_cmap('viridis')

# Get the total number of cycles to normalize the cycle number to [0, 1]
num_cycles = len(grouped_by_cycle)
norm = mcolors.Normalize(vmin=0, vmax=num_cycles)

# Create a plot
# Create a plot and specify axes
fig, ax = plt.subplots(figsize=(12, 8))

discharge_capacity = np.zeros(num_cycles)
cycle = np.zeros(num_cycles)
# Plot Voltage vs. Time for each cycle
for i, (cycle_number, group) in enumerate(grouped_by_cycle):
    D_cap = group['C_rate'].to_numpy()
    c_max = D_cap.max() # Assuming the last value represents the end of the cycle
    c_min = D_cap.min()   # Assuming the first value represents the start of the cycle
    
    V = group['V'].to_numpy()
    v_max = V.max() # Assuming the last value represents the end of the cycle
    v_min = V.min()    # Assuming the first value represents the start of the cycle
    
    discharge_capacity[i] = c_max  - c_min#(-np.diff(np.abs(V)) / np.diff(D_cap)).mean() #(v_max - v_min) / ((c_max - c_min) / 3600 )# Calculate the difference
    #discharge_capacity[i] = (v_max - v_min) / ((c_max - c_min)  )# Calculate the difference
    cycle[i] = cycle_number
    """
    D_cap = group['C_rate'].to_numpy()
    v_max = D_cap[-1]
    v_min = D_cap[0]
    #print(f"CMAX: {v_max} CMIN {v_min}")
    discharge_capacity[i] = v_min -  v_max 
    cycle[i] = cycle_number
    # Normalize the cycle_number for color selection
    """
colors = cmap(norm(cycle))
sc = plt.scatter(cycle, discharge_capacity, c=colors)
# Create a colorbar with the correct axes
cbar = plt.colorbar(sc, ax=ax, label='Normalized Cycle Number')
cbar.set_label('Cycle Number', rotation=270, labelpad=15)

plt.title('Diff(V)  / Diff(T) by cycle USSD')
plt.xlabel('cycle')
plt.ylabel('Diff(V)  / Diff(T)')
plt.show()

In [ ]:
from core import Wrap_PyBamm_Output
Wrap_PyBamm_Output.plot_av_at_by_cycle(df, 5).show()

In [ ]:
import matplotlib.pyplot as plt
filtered_df = df[df["Step"] == 4]
plt.scatter(df['SoC'], df['VoC'], c=df['Time'], alpha = 0.9, s = 0.1)
# Add labels and title
plt.title('V vs SoC Step 3')  # Plot Title
plt.xlabel('SoC (%)')  # X-axis Label
plt.ylabel('Voltage (V)')  # Y-axis Label
# Display the plot
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
import glob
import numpy as np
import pandas as pd

reduce_by = 1
version = "W10"
base_data_path="/home/ccanodom/data/EV_battery/" + version + "/" + version + "_" 
values = ["C02","C04","C13"]
def reduce_by(reduce_by):
    # Assuming df is your DataFrame
    # Step 1: Create a group identifier for each set of 20 rows
    group_identifier = np.arange(len(df)) // reduce_by

    # Step 2 & 3: Group by this identifier and calculate the mean for each group
    df_mean = df.groupby(group_identifier).mean()

    # If your DataFrame has a MultiIndex and you want to keep the non-numeric levels,
    # you may need to reset the index first, or handle them separately depending on your specific case.

    df_mean.reset_index(drop=True, inplace=True)
    return df_mean


list_df = []
for path in values:
    data_path = base_data_path + path + ".mat"
    print(f"Start with data in path {data_path}")
    df = EV_Data.get_data_EV(data_path)
    df_mean = reduce_by(30)
    
    list_df.append(df_mean)
df = pd.concat(list_df)
df = EV_Data.include_cycle_number(df)

In [ ]:
# Assuming df_adjusted is your DataFrame and has 'Step_Index' as a column
step_index_counts = df.reset_index()['Step_Index'].value_counts()

# Sorting the index to ensure the counts are displayed in order of the step index
step_index_counts = step_index_counts.sort_index()

# Display the counts
print(step_index_counts)

In [ ]:
df_mean.plot()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
df_discharge = df.xs(14, level='Step_Index')
# Group by 'Cycle_Number' if plotting for all cycles
grouped = df_discharge.groupby('Cycle_Number')
# Define the Greys colormap
cmap = cm.get_cmap('Reds')

# Get the total number of cycles to normalize the cycle number to [0, 1]
num_cycles = len(grouped)
norm = mcolors.Normalize(vmin=0, vmax=num_cycles)
# Plotting
plt.figure(figsize=(12, 8))

for cycle, group in grouped:

    time = group.index.get_level_values('Time')
    adjusted_time = time - time.min()
    color = cmap(norm(cycle))
    plt.plot(adjusted_time, group["Voltage"].diff()/adjusted_time.diff(), label=f'Cycle {cycle}', color=color, linewidth=0.1)
    plt.title("Values")
plt.xlabel("Time [s]")
plt.ylabel("Voltage")
plt.grid(True)
plt.show()

In [ ]:
# Example: Assuming USSD discharging corresponds to a specific range of Step_Index
# Adjust the step_index_range to match the actual USSD discharging step indices
ussd_discharging_step_index_range = (5, 9)  # This is an example range; adjust as necessary

# Filter the DataFrame for USSD discharging phase
ussd_discharge_df = df[(df.index.get_level_values('Step_Index') >= ussd_discharging_step_index_range[0]) &
                       (df.index.get_level_values('Step_Index') <= ussd_discharging_step_index_range[1])]

# Plot Voltage vs. Discharge Capacity for the filtered USSD discharging data
ussd_discharge_df.reset_index().plot(x='Discharge_Capacity', y='Voltage', kind='scatter', title='Voltage vs. Discharge Capacity (USSD Discharging)')
plt.xlabel('Discharge Capacity (Ah)')
plt.ylabel('Voltage (V)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import bokeh.io
%load_ext autoreload
%autoreload 2
# this is here only for completeness to clarify where
# the methods are nested (you probably already imported this earlier)


bokeh.io.reset_output()
bokeh.io.output_notebook()
    

In [ ]:

print(f"Before mean: {len(step_7_df)}. After {len(step_7_df.rolling(100).mean())}")

import numpy as np
import pandas as pd

# Assuming df is your DataFrame
# Step 1: Create a group identifier for each set of 20 rows
group_identifier = np.arange(len(df)) // 20

# Step 2 & 3: Group by this identifier and calculate the mean for each group
df_mean = df.groupby(group_identifier).mean()

# If your DataFrame has a MultiIndex and you want to keep the non-numeric levels,
# you may need to reset the index first, or handle them separately depending on your specific case.

df_mean.reset_index(drop=True, inplace=True)

# Display the first few rows of the downsampled DataFrame
print(df_mean.head())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Assuming df is already filtered for the USSD discharging phase
# Group the DataFrame by 'Cycle_Number' for plotting

grouped = df.groupby('Cycle_Number')

# Filter the DataFrame for Step_Index 7
step_7_df = df[df.index.get_level_values('Step_Index') == 9]

# Group the filtered DataFrame by 'Cycle_Number' for plotting

grouped_by_cycle = step_7_df.groupby('Cycle_Number')



# Assuming step_7_df and grouped_by_cycle are defined as before
# Define the Greys colormap
cmap = cm.get_cmap('Reds')

# Get the total number of cycles to normalize the cycle number to [0, 1]
num_cycles = len(grouped_by_cycle)
norm = mcolors.Normalize(vmin=0, vmax=num_cycles)

# Create a plot
plt.figure(figsize=(12, 8))

# Plot Voltage vs. Time for each cycle
for cycle_number, group in grouped_by_cycle:
    # Reset index to access the 'Time' column
    group = group.reset_index()
    # Adjust time so that each cycle's Step 7 starts at 0
    adjusted_time = group['Time'] - group['Time'].min()
    # Normalize the cycle_number for color selection
    color = cmap(norm(cycle_number))
    plt.plot(adjusted_time, group['Voltage'].rolling(100).mean(), label=f'Cycle {cycle_number}', color=color, linewidth=0.1)

plt.title('Voltage vs. Time for Step 9 by Cycle')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.show()

In [ ]:
from core import EV_Data
EV_Data.plot_voltage_vs_capacity(df, 14, "Discharge").plot()

In [ ]:
from core import EV_Data
EV_Data.plot_voltage_vs_capacity(df, 13, "Discharge").plot()

In [ ]:
EV_Data.plot_capacity_vs_time(df, 13, "discharge",3).plot()

In [ ]:
plot = EV_Data.plot_soc_vs_time_for_step(df, 4.8,2)

In [ ]:
plot.lenged()
plot.show()

In [ ]:
df["Charge_Capacity"].max() 

In [ ]:
df["Discharge_Capacity"].max() 

In [ ]:
# Create a plot
plt.figure(figsize=(10, 6))

for name, group in grouped:
    plt.plot(group['Discharge_Capacity'], group['Voltage'], label=f'Cycle {name}')

plt.title('Voltage vs. Discharge Capacity for USSD Discharging by Cycle')
plt.xlabel('Discharge Capacity (Ah)')
plt.ylabel('Voltage (V)')
plt.legend()  # Display a legend to identify each cycle
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extracting the specific arrays from the data
time = df.index.get_level_values(0).to_numpy()
charge_capacity = df['Charge_Capacity'].squeeze()
discharge_capacity = df['Discharge_Capacity'].squeeze()

# Plotting
plt.figure(figsize=(12, 6))

plt.plot(time, charge_capacity, label='Charge Capacity', color='blue')
plt.plot(time, discharge_capacity, label='Discharge Capacity', color='red')

plt.title('Charge/Discharge Capacity Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Capacity (Ah)') 
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
pd.DataFrame(step_index).plot()

In [ ]:
# Initialize a cycle number array with zeros, same length as step_index
cycle_numbers = np.zeros_like(step_index)

# Start with cycle number 1
current_cycle = 1

# Loop through the step index array to identify changes indicating a new cycle
for i in range(1, len(step_index)):
    # If the step index decreases, it indicates a new cycle has started
    if step_index[i] < step_index[i-1]:
        current_cycle += 1
    cycle_numbers[i] = current_cycle

# Checking the first few cycle numbers to verify our approach
cycle_numbers[:10], np.unique(cycle_numbers)

In [ ]:
import numpy as np

# Extract the step index data
step_index = data['Step_Index_full_vec_M1_NMC25degC'].squeeze()
voltage = data['V_full_vec_M1_NMC25degC'].squeeze()

# Find unique cycle indices to determine the first and last cycles
unique_cycles = np.unique(cycle_numbers)
first_cycle_index = unique_cycles[1]
last_cycle_index = unique_cycles[-1]

In [ ]:
# Identifying indices for the first and last cycle
first_cycle_indices = np.where(cycle_numbers == first_cycle_index)[0]
last_cycle_indices = np.where(cycle_numbers == last_cycle_index)[0]

# Extracting data for the first cycle
voltage_first_cycle = voltage[first_cycle_indices]
charge_capacity_first_cycle = charge_capacity[first_cycle_indices]

# Extracting data for the last cycle
voltage_last_cycle = voltage[last_cycle_indices]
charge_capacity_last_cycle = charge_capacity[last_cycle_indices]

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# First cycle plot
axs[0].plot(charge_capacity_first_cycle, voltage_first_cycle, label='First Cycle', color='blue')
axs[0].set_title('Voltage vs. Capacity (First Cycle)')
axs[0].set_xlabel('Capacity (Ah)')
axs[0].set_ylabel('Voltage (V)')
axs[0].grid(True)
axs[0].legend()

# Last cycle plot
axs[1].plot(charge_capacity_last_cycle, voltage_last_cycle, label='Last Cycle', color='red')
axs[1].set_title('Voltage vs. Capacity (Last Cycle)')
axs[1].set_xlabel('Capacity (Ah)')
axs[1].set_ylabel('Voltage (V)')
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame(cycle_numbers).plot()

In [ ]:
# Extracting discharge capacity data
discharge_capacity = data['dis_cap_full_vec_M1_NMC25degC'].squeeze()

# Identifying indices for the third and the last cycle
third_cycle_indices = np.where(cycle_numbers == 3)[0]
last_cycle_indices = np.where(cycle_numbers == np.max(cycle_numbers))[0]

# Extracting data for the third cycle
voltage_third_cycle = voltage[third_cycle_indices]
discharge_capacity_third_cycle = discharge_capacity[third_cycle_indices]

# Extracting data for the last cycle
voltage_last_cycle = voltage[last_cycle_indices]
discharge_capacity_last_cycle = discharge_capacity[last_cycle_indices]

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# Third cycle plot
axs[0].plot(discharge_capacity_third_cycle, voltage_third_cycle, label='Third Cycle', color='blue')
axs[0].set_title('Voltage vs. Discharge Capacity (Third Cycle)')
axs[0].set_xlabel('Discharge Capacity (Ah)')
axs[0].set_ylabel('Voltage (V)')
axs[0].grid(True)
axs[0].legend()

# Last cycle plot
axs[1].plot(discharge_capacity_last_cycle, voltage_last_cycle, label='Last Cycle', color='red')
axs[1].set_title('Voltage vs. Discharge Capacity (Last Cycle)')
axs[1].set_xlabel('Discharge Capacity (Ah)')
axs[1].set_ylabel('Voltage (V)')
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for cycle 4
cycle_4_df = df.xs(1, level='Cycle_Number')

# Separate the data for plotting
time = cycle_4_df.reset_index()['Time']
voltage = cycle_4_df['Voltage']
step_index = cycle_4_df.reset_index()['Step_Index']

# Create the plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Voltage on the primary y-axis
color = 'tab:red'
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Voltage (V)', color=color)
ax1.plot(time, voltage, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a secondary y-axis for the Step Index
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('Step Index', color=color)  
ax2.plot(time, step_index, color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Add a title and show the plot
plt.title('Cycle 4: Voltage and Step Index vs. Time')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for cycle 4
cycle_4_df = df.xs(1, level='Cycle_Number')

# Separate the data for plotting
time = cycle_4_df.reset_index()['Time']
voltage = cycle_4_df['Voltage']
step_index = cycle_4_df['Current']

# Create the plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Voltage on the primary y-axis
color = 'tab:red'
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Voltage (V)', color=color)
ax1.plot(time, voltage, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a secondary y-axis for the Step Index
ax2 = ax1.twinx()  
color = 'tab:green'
ax2.set_ylabel('Current (A)', color=color)  
ax2.plot(time, step_index, color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Add a title and show the plot
plt.title('Cycle 4: Voltage and Step Index vs. Time')
plt.show()